In [ ]:
import tensorflow.keras as keras
import numpy as np
import importlib

import kaggle_functions as kaggle
import model10_wideresnet
importlib.reload(kaggle); 
importlib.reload(model11_stacking);
from model11_stacking import Model

model_number = 'model11'

### Create datasets

In [2]:
train_dataset, valid_dataset, valid_labels = kaggle.load_train_as_dataset()

batch_size = 32
train_dataset_augmented, epoch_length = kaggle.augment_dataset(train_dataset, batch_size)

In [ ]:
kaggle.show_images(train_dataset_augmented, 9)

### Train new model

In [ ]:
Model().build(None).summary()

In [57]:
# Fit model
# See {model}.py for specific training instructions, like epochs or valid_patience
model_name = 'Stacking_0'

model, history = kaggle.train_model(
    Model().build(None), train_dataset.batch(32), valid_dataset, 
    epochs=10, valid_patience=3, epoch_length=epoch_length)

Epoch 1/10
298/298 [==============================] - 141s 451ms/step - loss: 2.8242 - accuracy: 0.5981 - val_loss: 2.0878 - val_accuracy: 0.8318 - lr: 5.0000e-05
Epoch 2/10
298/298 [==============================] - 131s 441ms/step - loss: 1.8409 - accuracy: 0.9333 - val_loss: 2.0137 - val_accuracy: 0.8583 - lr: 5.0000e-05
Epoch 3/10
298/298 [==============================] - 129s 431ms/step - loss: 1.6936 - accuracy: 0.9691 - val_loss: 2.0216 - val_accuracy: 0.8688 - lr: 5.0000e-05
Epoch 4/10
298/298 [==============================] - 128s 431ms/step - loss: 1.6398 - accuracy: 0.9831 - val_loss: 2.0337 - val_accuracy: 0.8701 - lr: 5.0000e-05
Epoch 5/10
298/298 [==============================] - 129s 433ms/step - loss: 1.6192 - accuracy: 0.9843 - val_loss: 2.0507 - val_accuracy: 0.8717 - lr: 5.0000e-05
Epoch 6/10
298/298 [==============================] - 129s 433ms/step - loss: 1.6044 - accuracy: 0.9875 - val_loss: 2.0652 - val_accuracy: 0.8734 - lr: 5.0000e-05
Epoch 7/10
298/298 [==

In [ ]:
# Save model
model.save(f'models/{model_number}/{model_name}')

# Plot model statistics during training
kaggle.plot_model_history(history, [['accuracy', 'val_accuracy'], ['loss', 'val_loss']])

### Fine-tune

In [9]:
# Fine-tune model
fine_model, history = kaggle.fine_tune_model(
    model,
    train_dataset.batch(32), valid_dataset, 
    epochs=2, epoch_length=epoch_length, learning_rate=1E-5)

Epoch 1/2
298/298 [==============================] - 143s 458ms/step - loss: 1.7360 - accuracy: 0.9830 - val_loss: 2.3162 - val_accuracy: 0.8764
Epoch 2/2
298/298 [==============================] - 134s 451ms/step - loss: 1.7383 - accuracy: 0.9842 - val_loss: 2.3240 - val_accuracy: 0.8751


In [16]:
# Save fine-tuned model
fine_model_name = 'WideResNet_5_84'
fine_model.save(f'models/{model_number}/{fine_model_name}')

INFO:tensorflow:Assets written to: models/model10/WideResNet_5_84\assets


C:\Users\Guillaume\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
C:\Users\Guillaume\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\saving\saved_model\layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


### Evaluate

In [ ]:
model_name = ''
model = keras.models.load_model(f'models/{model_number}/{model_name}')
test_pred = np.argmax(model.predict(valid_dataset.batch(128)), axis=1)

kaggle.print_accuracy(valid_labels, test_pred)
kaggle.plot_confusion_matrix(valid_labels, test_pred)

### Generate test predictions

In [ ]:
model_name = 'WideResNet_3'
kaggle.generate_test_pred_filepath(f'models/{model_number}/{model_name}')

In [11]:
model_name = 'Stacking_0'
kaggle.generate_test_pred(model, f'models/{model_number}/{model_name}_test_pred.csv')

### Hypertune models

In [ ]:
tuner = kaggle.hypertune_model(
    Model(), train_dataset.batch(32).cache(), valid_dataset, 
    model_number, 'hypertuner2021-12-03', trials=4, 
    epochs=11, valid_patience=3, epoch_length=epoch_length)

In [4]:
tuner = kaggle.load_hypertuner(Model(), model_number, 'hypertuner2021-12-03')

INFO:tensorflow:Reloading Oracle from existing project models/model11\hypertuner2021-12-03\oracle.json
INFO:tensorflow:Reloading Tuner from models/model11\hypertuner2021-12-03\tuner0.json
Results summary
Results in models/model11\hypertuner2021-12-03
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
dropout: 0.3
l2_reg: 0.001
Score: 0.8767872452735901
Trial summary
Hyperparameters:
dropout: 0.3
l2_reg: 0.0001
Score: 0.8738435506820679
Trial summary
Hyperparameters:
dropout: 0.6
l2_reg: 0.001
Score: 0.8717409372329712
Trial summary
Hyperparameters:
dropout: 0.6
l2_reg: 0.0001
Score: 0.8708999156951904


In [10]:
model = tuner.get_best_models(1)[0]